# Markov Chains (local version)
Department of Computer Science    
University of York

Authors: Dr David Zendle and Dr James Stovold

Download original file from https://github.com/jstovold/UoY-POVD
***

## Introduction 

This is a Jupyter notebook.

It is an interactive environment that lets you both write and execute python scripts in a virtual environment.

If you look to the left of the page, there should be a blue or green vertical bar. This shows you which chunk of code is currently selected. 

If you click on the following chunk of code, you will see that the vertical bar moves down. Click the 'Run' button at the top of the screen to execute that code.

In [ ]:
print("for instance, pressing run while this code is selected will print this text out")

If you want to change the code, ensure the bar on the left is green by clicking in the code itself, then you are able to make changes. Why not try this out by changing the code below to output the number 5 instead of 10?

In [ ]:
i=10
print("or running this code will print out the number",i)

***
Markov chains are mathematical systems that experience changes from one state to another according to certain probabilistic rules. Markov chains have lots of practical uses: For example, they can be used for predictive text on a mobile device.

I've set up this notebook to show you how markov chains can be used to generate text. You can learn how this works by stepping through the program, reading each piece of text and pressing the 'play' symbol next to the associated code.

When you've seen how the code works, there are a few challenges for you: (N.B. The following Gutenberg links won't work from Germany)

1. Can you get it to generate text from a different book? At the moment, the text generator is working from Jane Austen's Pride and Prejudice, located in a .txt file at Project Gutenberg ('[http://www.gutenberg.org/ebooks/42671.txt.utf-8](http://www.gutenberg.org/ebooks/42671.txt.utf-8)').  However, it should theoretically be able to generate text from any source file on the internet - for example Dracula ('[http://www.gutenberg.org/ebooks/345.txt.utf-8](http://www.gutenberg.org/ebooks/345.txt.utf-8)') or something called 'Astounding Stories of Super-Science' ('[https://www.gutenberg.org/ebooks/29768.txt.utf-8](https://www.gutenberg.org/ebooks/29768.txt.utf-8)'). I'm using old books because you can find them easily online (copyright free) from Project Gutenberg (e.g. [https://www.gutenberg.org/wiki/Science_Fiction_(Bookshelf)](https://www.gutenberg.org/wiki/Science_Fiction_(Bookshelf) )). But you could use anything.

2. Can you get it to work with something bigger or smaller than a 3-gram? Say a 4-gram or a 2-gram?

3. Can you get it to create a mash-up of two books? This might involve creating a second bookInput from a different url, and then stitching the resulting strings together.

## Now, for the code itself...

First, load a text file from Project Gutenberg to use as input for our text generator.

The URL here points to a .txt file of Jane Austen's Pride and Prejudice.

That URL is downloaded, decoded, and stored in a string variable called ```bookInput```.

### Germany-specific information

Due to a copyright lawsuit (over 18 books!) Project Gutenberg is not accessible from within Germany. To get round this problem, the following piece of code will load Pride and Prejudice into memory from a local copy. If you wish to change the book used, Dracula is available at 'dracula.txt' and Astounding Stories... is available at 'astounding.txt'. 

It is worth highlighting that the books we are using are not part of the 18 books in the lawsuit, and are free to use worldwide. 

In [ ]:
import random
import urllib
import textwrap
# non-germany-specific version:
# bookInput = urllib.request.urlopen('http://www.gutenberg.org/ebooks/42671.txt.utf-8').read().decode('utf-8')
# print(bookInput)

# germany-specific version:
f = open('pride.txt', 'r')
bookInput = f.read()
print(bookInput)
f.close()

Now, the next thing we want to do is split our file up into tokens that we can use for our Markov chain's 'states'. 

The easiest way to do this is just to split our book up into words using python's 'split' command. This creates a list of each word in the book

In [ ]:
wordList = bookInput.split()
print("The number of words in the book are:",(len(wordList)))
print(wordList)

The next thing we want to do is take that big list, and transform it into a Markov Chain representation.

Remember, Markov Chains are systems that shift from states according to probabilistic rules.

So, if we had the sequence: A B B C, we would know that A->B 100% of the time; B->B 50% of the time, B->C 50% of the time. We want to form a similar probabilistic model here, but using words. Something that tells us "If the current word is Pride, there is a 80% chance of going to the word 'and', a 10% chance of going to the word 'of', etc. etc. etc."

The simplest way to do this is a dictionary: This will be a data structure that holds a mapping between each word, and all the words that ever follow that word. For example, if we had the sentence "A sailor went to sea to see what he could see", it would become the following dictionary:

----------------
A->sailor

sailor->went

went->to

to->sea,see

sea->to

what->he

he->could

could->see

------------------

First, we create a set from each unique word in our list.

Then, we use that set as keys to a dictionary, with each word linked to a (currently) empty list of 'following' words, which is denoted in python as '[]'




In [ ]:
wordSet = set(wordList)
print("There are this many unique words in the list: ",len(wordSet))
wordDictionary = dict((word,[]) for word in wordSet)


Now, it's time to propogate the predictive model embodied in our dictionary with the actual data from our book.

We use a 'for' loop to step through our list of words. For each word in our book, we record the next word after it in our dictionary. 

In [ ]:
for i in range(0,len(wordList)-1):
    currentWord = wordList[i]
    nextWord = wordList[i+1]
    wordDictionary[currentWord].append(nextWord)
    
print("Model is built!")

Let's see if it works! What words come after the word 'Darcy'?

In [ ]:
if "Darcy" in wordDictionary:
    print(wordDictionary["Darcy"])

Here's where the fun begins. We've successfully built a predictive model: Now let's have it try to generate some text.

Let's start with a random word from our list, and randomly pick one of the words that it links to; then for that word, let's randomly pick one of the words that *it* links to; and so on!

The only tricky bit of code here is building in a safeguard for a situation where our current word doesn't link to anything else: If this is the case, we break out of the 'for' loop and end our story early.

In [ ]:
story = ""
word = random.choice(wordList)
for i in range (0,100):
    story = story +" "+ word
    possibleNextWords = wordDictionary[word]
    if len(possibleNextWords)==0:
        break
    word = random.choice(possibleNextWords)

for line in textwrap.wrap(story):
    print(line)




So, that's kind of generating text. It's not very coherent, though.

How can we improve our generative model?

A quick improvement can be made via leveraging something called an n-gram: Basically, by chunking our input into fewer states, we can make our output obey implicit grammatical rules.

For example, the input "The cat sat in the hat" could be represented as:

---------
The->cat

cat->sat

sat->in

in->the

the->hat

---------

Or it could be represented as the following 2-grams:

---------

The cat->sat in

cat sat->in the

sat in->the hat


---------

Note how the second contains implicit knowledge about things like articles of speech (e.g. 'the hat')

We could even represent this through 3-grams:

---------

The cat sat -> in the hat

---------

Note that the larger your gram size, though, the more data you need to build your model.

Let's take our word list input, and transform it into a list of 3-grams

In [ ]:
gramSize = 3
gramList=[]

for i in range(0,len(wordList)-(gramSize-1)):
    gram = wordList[i]
    for j in range(1,gramSize):
        gram = gram+" "+wordList[i+j]
    gramList.append(gram)  

print(gramList)


Now we can use that list of 3-grams to create a set of 3-grams, and a dictionary of 3-grams, just like we did with individuals words

In [ ]:
gramSet = set(gramList)
print("There are this many unique grams in the list: ",len(gramSet))
gramDictionary = dict((gram,[]) for gram in gramSet)

for i in range(0,len(gramList)-gramSize):
    currentGram = gramList[i]
    nextGram = gramList[i+gramSize]
    gramDictionary[currentGram].append(nextGram)


Now, let's try generating some text from this model using exactly the same code as before...

In [ ]:
story = ""
gram = random.choice(gramList)
for i in range (0,100):
    story = story +" "+ gram
    possibleNextGrams = gramDictionary[gram]
    if len(possibleNextGrams)==0:
        break
    gram = random.choice(possibleNextGrams)

for line in textwrap.wrap(story):
    print(line)

Much more coherent!

Now, if you're interested, go to the top and do the challenges.

Note that if you want to run all the code again, you don't have to step through every single line.

The menu at the top (Cell->Run All) will just run all the code for you! You can then just scroll to the bottom to see the end product. You might have to wait a little while for the computation to complete, though.